In [ ]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET

import pandas as pd
import numpy as np

from future.utils import iteritems
from pyPRMS.prms_helpers import float_to_str

In [ ]:
# Manually generated control file parameters from setup.c
workfile = '<path>/PRMS6/prms_vars_raw.txt'
df = pd.read_csv(workfile, sep='\t')

df.head()

### Generate a XML file for parameters

In [ ]:
# <variable name="dprst_area_clos_max">
#     <type>F</type>
#     <units>acres</units>
#     <desc>Aggregate sum of closed ...</desc>
#     <modules>
#         <module>srunoff</module>
#     </modules>
#     <dimensions>
#         <dimension>nhru</dimension>
#     </dimensions>
# </variable>

NHM_DATATYPES = {'I': 'integer', 'F': 'real', 'D': 'double', 'S': 'string'}
inv_map = {vv: kk for kk, vv in iteritems(NHM_DATATYPES)}

params_xml = xmlET.Element('variables')

for index, row in df.iterrows():
#     print(row)
    param_sub = xmlET.SubElement(params_xml, 'variable')
    param_sub.set('name', row['variable'])

    xmlET.SubElement(param_sub, 'type').text = inv_map[row['datatype']]

    xmlET.SubElement(param_sub, 'units').text = row['units']
    xmlET.SubElement(param_sub, 'desc').text = row['description']
            
    modules_sub = xmlET.SubElement(param_sub, 'modules')

    for mm in (row['modules']).split(','):
        xmlET.SubElement(modules_sub, 'module').text = mm
        
    dimensions_sub = xmlET.SubElement(param_sub, 'dimensions')
    
    for ii, dd in enumerate((row['dimensions']).split(',')):
        curr_dim = xmlET.SubElement(dimensions_sub, 'dimension')
        curr_dim.set('name', dd)
        xmlET.SubElement(curr_dim, 'position').text = str(ii+1)
 

In [ ]:
xmlstr = minidom.parseString(xmlET.tostring(params_xml)).toprettyxml(indent='    ')

with open('<path>/tmp/variables.xml', 'w') as ff:
    ff.write(xmlstr)

In [ ]:
df